In [11]:
from utils.data_gatherer import *
import matplotlib.pyplot as plt
import numpy as np
import unicodedata

import warnings
warnings.filterwarnings('ignore')

1. Get daily data

In [20]:
symbol = 'خودرو'
price_df  = get_daily_price_data(symbol= symbol, start_date='1400-01-01', end_date='1403-09-01')

In [21]:
price_df.head()

,jdate,Date,High,Low,Final,Close,Open,Y-Final,Value,Volume,No,Weekday,Adj Open,Adj High,Adj Low,Adj Close,Adj Final
4570,1400-01-07,2021-03-27,2690,2490,2630,2670,2520,2540,-2147483648,878226906,28283,Saturday,2520,2690,2490,2670,2630
4571,1400-01-08,2021-03-28,2730,2590,2650,2610,2670,2630,-2147483648,343600591,13933,Sunday,2670,2730,2590,2610,2650
4572,1400-01-10,2021-03-30,2640,2600,2610,2600,2640,2650,-2147483648,123895903,5127,Tuesday,2640,2640,2600,2600,2610
4573,1400-01-11,2021-03-31,2580,2560,2560,2560,2560,2610,-2147483648,78264146,2659,Wednesday,2560,2580,2560,2560,2560
4574,1400-01-14,2021-04-03,2510,2510,2540,2510,2510,2560,-2147483648,22306197,962,Saturday,2510,2510,2510,2510,2540


In [66]:
yearly_days = 252
path_num = 10
steps_num = 120
dt = 1/52

past_price = price_df['Adj Close'].iloc[:-steps_num]
foward_price = price_df['Adj Close'].iloc[-steps_num:]

return_series = np.diff(np.log(past_price))
mu = return_series * yearly_days
sigma = np.std(return_series) * yearly_days ** (0.5)

random_array = np.random.randn(path_num, steps_num)
simulated_price_array = np.zeros((path_num, steps_num))

In [70]:
simulated_price_array[:, 0] = past_price.iloc[-1]
for i in range(steps_num-1):
    random_array[:, i] = (random_array[:, i] - np.mean(random_array[:, i])) / np.std(random_array[:, i])
    simulated_price_array[:, i+1] = simulated_price_array[:, i] * np.exp(((mu-sigma**2/2)) * dt + sigma * np.sqrt(dt) * random_array[:, i])

ValueError: operands could not be broadcast together with shapes (634,) (10,) 

2. Price path params